In [10]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import pickle

In [2]:
def read_df(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)
        
        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)
        
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda tf: tf.total_seconds() / 60)
    
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    cat = ['PULocationID', 'DOLocationID']
    df[cat] = df[cat].astype(str)
    return df

In [3]:
df_train = read_df('data/yellow_tripdata_2023-01.parquet')
df_valid = read_df('data/yellow_tripdata_2023-02.parquet')

In [4]:
cat = ['PULocationID', 'DOLocationID']
train_dict = df_train[cat].to_dict(orient='records')
valid_dict = df_valid[cat].to_dict(orient='records')

In [5]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dict)
X_valid = dv.transform(valid_dict)

In [6]:
target = 'duration'
y_train = df_train[target].values
y_valid = df_valid[target].values

In [7]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_train)

In [8]:
mean_squared_error(y_true=y_train, y_pred=y_pred, squared=False)

7.649261027792376

In [9]:
y_pred_valid = lr.predict(X_valid)
mean_squared_error(y_true=y_valid, y_pred=y_pred_valid, squared=False)

7.811832836304415

In [11]:
with open('model/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)